###MLIB Techniques

-Data Frame API

#Base upon age and experience predict the salary

* Spark MLib there are many examples
* Data Frame API and RDD techniques
* We will be learning Data Frame API in thi session

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/12 13:59:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
## Read the datset
training = spark.read.csv("Employee_Salary_Dataset.csv", header= True, inferSchema= True)

In [5]:
training.show(5)

+---+----------------+---+------+------+
| ID|Experience_Years|Age|Gender|Salary|
+---+----------------+---+------+------+
|  1|               5| 28|Female|250000|
|  2|               1| 21|  Male| 50000|
|  3|               3| 23|Female|170000|
|  4|               2| 22|  Male| 25000|
|  5|               1| 17|  Male| 10000|
+---+----------------+---+------+------+
only showing top 5 rows



In [6]:
training.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Experience_Years: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['ID', 'Experience_Years', 'Age', 'Gender', 'Salary']

In [10]:
!pip install numpy

     |████████████████████████████████| 17.3 MB 23.3 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## In pyspark we create a way to group of independent features. We create a vector assembler.
* In general in ML we do train and test split.
* We create dependent and independent features.
* In the above example [Experience_Years', 'Age'] are independent features. 
* In pyspark we group independent features into one new features and treat this new feature as independent features
* If character feature is there conver it into numerical feature

In [12]:
from pyspark.ml.feature import  VectorAssembler
featureassembler = VectorAssembler(inputCols=['Experience_Years', 'Age'], outputCol="Independent Features")

In [13]:
output = featureassembler.transform(training)

In [15]:
output.show(5)

+---+----------------+---+------+------+--------------------+
| ID|Experience_Years|Age|Gender|Salary|Independent Features|
+---+----------------+---+------+------+--------------------+
|  1|               5| 28|Female|250000|          [5.0,28.0]|
|  2|               1| 21|  Male| 50000|          [1.0,21.0]|
|  3|               3| 23|Female|170000|          [3.0,23.0]|
|  4|               2| 22|  Male| 25000|          [2.0,22.0]|
|  5|               1| 17|  Male| 10000|          [1.0,17.0]|
+---+----------------+---+------+------+--------------------+
only showing top 5 rows



## Independent Features is my input feature and Salary is my dependent features

In [16]:
output.columns

['ID', 'Experience_Years', 'Age', 'Gender', 'Salary', 'Independent Features']

In [19]:
finalized_data = output.select( 'Independent Features','Salary',)

In [21]:
finalized_data.show(10)

+--------------------+-------+
|Independent Features| Salary|
+--------------------+-------+
|          [5.0,28.0]| 250000|
|          [1.0,21.0]|  50000|
|          [3.0,23.0]| 170000|
|          [2.0,22.0]|  25000|
|          [1.0,17.0]|  10000|
|         [25.0,62.0]|5001000|
|         [19.0,54.0]| 800000|
|          [2.0,21.0]|   9000|
|         [10.0,36.0]|  61500|
|         [15.0,54.0]| 650000|
+--------------------+-------+
only showing top 10 rows



In [23]:
### train test split using Linear Regression 
from pyspark.ml.regression import  LinearRegression


In [24]:
train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

23/07/12 14:52:19 WARN Instrumentation: [ae59d7f2] regParam is zero, which might cause numerical instability and overfitting.
23/07/12 14:52:21 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/07/12 14:52:21 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/07/12 14:52:21 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [25]:
### Coefficients 
regressor.coefficients

DenseVector([-77618.4914, 165369.2774])

In [26]:
### Intercepts 
regressor.intercept

-3284623.7954826253

In [28]:
### Predictions 
pred_results = regressor.evaluate(test_data)

In [29]:
pred_results.predictions.show()

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+--------+-------------------+
|Independent Features|  Salary|         prediction|
+--------------------+--------+-------------------+
|          [1.0,18.0]|    3000|-385595.29397666734|
|          [2.0,21.0]|    9000|  32894.04673536727|
|          [3.0,22.0]|   20000| 120644.83267487865|
|         [15.0,54.0]|  900000|  4481039.811674248|
|         [16.0,49.0]| 7600000| 3576574.9332961896|
|         [19.0,54.0]| 5000000| 4170565.8458872484|
|         [25.0,62.0]| 5001000|  5027809.116296841|
|         [27.0,62.0]|10000000|   4872572.13340334|
+--------------------+--------+-------------------+



In [31]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1762658.7735964032, 7019087285296.024)